In [1]:
from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, TwitterPager
import pandas as pd
import tweepy
import time

In [2]:
consumer_key= "oJPfWgPQWoTmmcDV8wt7XQiJm"
consumer_secret= "4ievAUWvc87v6wgOh7IvqTVtvLnJ6esvvZjfkVTgy62g8MvvHL"
access_token_key= "1480765674532192257-JCXhWc1t7QtEmLBKMtsQ9sqZFhPaqy"
access_token_secret= "kmBKSuixBf9FimkuAvlbQd372rf4criee246H36MUQdsy"
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret, api_version='2')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
apis = tweepy.API(auth)

In [ ]:
names = ["WatcherGuru"]  # placeholder screen name for the handle you want to retrieve tweets from
# YOU CAN QUERY MULTIPLE HANDLES, BUT REMEMBER TO HANDLE TWITTER'S TIMEOUT/LIMIT CONSTRAINTS

""" Retrieves the most recent 200-ish tweets from a  list of given screen names
    Returns a data frame of the tweets with [id, screen_name, text, timestamp]
"""
def get_new_tweets(names):
    print("Retrieving tweets")
    corpus = []                                                                                        
    for name in names:
        tweets = apis.user_timeline(name, include_rts=False, count=200, tweet_mode="extended")          
        time.sleep(4)
        corpus.extend(tweets)                                                                          
    data = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at] for tweet in corpus]
    tweets = pd.DataFrame(data, columns=['tweet_id', 'screen_name', 'text', 'timestamp'])                

    return tweets

In [ ]:
""" Adds conversation_ids to the tweets retrieved from get_new_tweets
    Returns a data frame of the tweets with [id, screen_name, text, timestamp, conversation_id]
"""
def add_data(tweets):
    print("Retrieving additional data")
    ids = tweets.tweet_id
    conv_ids = []
    for id in ids:

        TWEET_ID = id
        TWEET_FIELDS = 'conversation_id'

        try:
            r = api.request(f'tweets/:{TWEET_ID}', {'tweet.fields': TWEET_FIELDS})

            for item in r:
                conv_ids.append(item['conversation_id'])


        except TwitterRequestError as e:
            print(e.status_code)
            for msg in iter(e):
                print(msg)

        except TwitterConnectionError as e:
            print(e)

        except Exception as e:
            print(e)

    tweets['conversation_id'] = conv_ids
    return tweets


In [ ]:
# RETRIEVE TWEETS FROM THE SCREEN NAMES(S)
tweets = get_new_tweets(names)
# RETRIEVE CONVERSATION IDs OF THE RETRIEVED TWEETS
tweets = add_data(tweets)
# WRITE THE RETRIEVED TWEETS TO CSV
tweets.to_csv("tweets.csv")
# VIEW THE DATAFRAME HEAD WITH THE TWEETS RETRIEVED
tweets.head()

In [ ]:
unq_conv = list(set(tweets["conversation_id"].to_list()))

In [3]:
conv_ids = unq_conv
# replace this with conversation IDs from above- this is just a place holder

In [4]:
class TreeNode:
	def __init__(self, data):
		"""data is a tweet's json object"""
		self.data = data
		self.children = []

	def id(self):
		"""a node is identified by its author"""
		return self.data['author_id']

	def reply_to(self):
		"""the reply-to user is the parent of the node"""
		return self.data['in_reply_to_user_id']

	def find_parent_of(self, node):
		"""append a node to the children of it's reply-to user"""
		if node.reply_to() == self.id():
			self.children.append(node)
			return True
		for child in self.children:
			if child.find_parent_of(node):
				return True
		return False

	def print_tree(self, level):
		"""level 0 is the root node, then incremented for subsequent generations"""
		# print(f'{level*"_"}{level}: {self.id()}')
		level += 1
		for child in self.children:
			child.print_tree(level)

	def list_l1(self):
		conv_id = []
		child_id = []
		text = []
		# print(self.data['id'])
		for child in self.children:
			#print(child.data)
			conv_id.append(self.data['id'])
			child_id.append(child.data['author_id'])
			text.append(child.data['text'])
		#print(conv_id, child_id, text)
		return conv_id, child_id, text

In [5]:
"""
Retrieves level 1 replies for a given conversation id
Returns lists conv_id, child_id, text tuple which shows every reply's tweet_id and text in the last two lists
"""
def retrieve_replies(conversation_id):
    try:
        # GET ROOT OF THE CONVERSATION
        r = api.request(f'tweets/:{conversation_id}',
                        {
                            'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                        })

        for item in r:
            root = TreeNode(item)
            # print(f'ROOT {root.id()}')

        # GET ALL REPLIES IN CONVERSATION

        pager = TwitterPager(api, 'tweets/search/recent',
                             {
                                 'query': f'conversation_id:{conversation_id}',
                                 'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                             })

        orphans = []

        for item in pager.get_iterator(wait=2):
            node = TreeNode(item)
            # print(f'{node.id()} => {node.reply_to()}')
            # COLLECT ANY ORPHANS THAT ARE NODE'S CHILD
            orphans = [orphan for orphan in orphans if not node.find_parent_of(orphan)]
            # IF NODE CANNOT BE PLACED IN TREE, ORPHAN IT UNTIL ITS PARENT IS FOUND
            if not root.find_parent_of(node):
                orphans.append(node)

        conv_id, child_id, text = root.list_l1()
#         print('\nTREE...')
# 	    root.print_tree(0)

        assert len(orphans) == 0, f'{len(orphans)} orphaned tweets'

    except TwitterRequestError as e:
        print(e.status_code)
        for msg in iter(e):
            print(msg)

    except TwitterConnectionError as e:
        print(e)

    except Exception as e:
        print(e)

    return conv_id, child_id, text



"""
Retrieves replies for a list of conversation ids (conv_ids
Returns a dataframe with columns [conv_id, child_id, text] tuple which shows every reply's tweet_id and text in the last two columns
"""
def reply_thread_maker(conv_ids):
    conv_id = []
    child_id = []
    text = []
    for id in conv_ids:
        print(id)
        conv_id1, child_id1, text1 = retrieve_replies(id)
        conv_id.extend(conv_id1)
        child_id.extend(child_id1)
        text.extend(text1)

    replies_data = {'conversation_id' : conv_id,
               'child_tweet_id': child_id,
               'tweet_text' : text}

    replies= pd.DataFrame(replies_data)
    return replies


In [6]:
replies = reply_thread_maker(conv_ids)

replies["Screen_Name"] = ""
for i, reply in replies.iterrows():
    
    api = tweepy.API(auth)
    try:
        print("Try", reply["child_tweet_id"])
        user = api.get_user(reply["child_tweet_id"])
        replies["Screen_Name"][i] = user.screen_name
    except:
        print("Except", reply["child_tweet_id"])
        continue
# WRITE REPLIES TO FILE


1482535481267085318
1 orphaned tweets
Try 1477920969838661632
Try 1363137144173129740
Try 39923673
Try 933761707754713088
Try 1470145355236978698
Try 1457084698274971650
Try 1395692935623151618
Try 1475542466766032897
Try 1464585270167277568
Try 1478256109286825984
Try 1480482266313031680
Try 1302244935857512448
Try 1477603794552766466
Try 1455835756329537539
Try 1446554151983460359
Try 901296386012057600
Try 1477400458952118350
Try 67974248
Try 1454764033253457922
Try 417530297
Try 1403931270833262594
Try 1481975111553007616
Try 1471550478131470338
Try 49739715
Try 1430743879897124873
Try 1453033526782275590
Try 1297628426522812418
Try 1304166174238834693
Try 78319742
Try 1392194660647510023
Try 1360195201650851842
Try 1166680085627375616
Try 1383351468065918985
Try 1387354622092845056
Try 1481959299157196802
Try 1451487392188321794
Try 1424011398095327236
Try 1426156397994065927
Try 1482161033490100225
Try 4559880886
Try 1424682502220881921
Try 2861131850
Try 1482411240240656384
Try 

Try 1470371689036869633
Try 1144953489094660097
Try 1350149806795190272
Try 1464447979268493315
Try 818997022074146817
Try 1461599009261244416
Try 1160919476587700352
Try 1361803981039755266
Try 1542417734
Try 1438635592363872262
Try 1481412932042838016
Try 1390765155810750465
Try 1476069545223749633
Try 1438607328903933957
Try 399922821
Try 1476291432113709066
Try 1250132560186953735
Try 1470029454655340544
Try 1250132560186953735
Try 1452912436206608384
Try 955146747902181376
Try 955146747902181376
Try 1470029454655340544
Try 1289362545334079489
Try 1470029454655340544
Try 1395216201047973888
Try 1470029454655340544
Try 1468939801864773632
Try 1458124987420790787
Try 1226992941954146305
Try 1431078094178242563
Try 1481305216960008194
Try 951140437200375809
Try 1436851865090723841
Try 1399571552841129986
Try 1377797121131286535
Try 1294144532460613633
Try 1477960737322672132
Try 750053662769180672
Try 1143260941871734786
Try 1143260941871734786


In [7]:
replies.to_csv("replies_"+str(conv_ids)+".csv")
#VIEW SAMPLE REPLIES
replies.head()

,conversation_id,child_tweet_id,tweet_text,Screen_Name
0,1482535481267085318,1477920969838661632,@WatcherGuru #kda,kamal_taghipour
1,1482535481267085318,1363137144173129740,@WatcherGuru With #crypto financial freedom is...,mistywhite33
2,1482535481267085318,39923673,@WatcherGuru 🔥Most desired Mystery Game is lau...,juslilbit87
3,1482535481267085318,933761707754713088,@WatcherGuru ⛏💎DIAMOND MINE TOKEN💎⛏\n\n🔥WILL L...,Annelloxxe
4,1482535481267085318,1470145355236978698,@WatcherGuru BREAKING: @BanxaOfficial NASDAQ L...,becktoham


In [ ]:
replies["conversation_id"].unique()

In [ ]:
tweets["conversation_id"].unique()